# Tile Stitching : Merging unmixed files to OME-TIFF


[![View on GitHub](https://img.shields.io/badge/View-on%20GitHub-lightgrey?logo=github)](https://github.com/Dana-Farber-AIOS/pathml/blob/master/examples/)

The field of digital pathology often involves the management and processing of large, high-resolution images, particularly those in the form of tiled image sets. For instance, in whole-slide imaging (WSI), an entire microscope slide is scanned at high resolution, often creating gigapixel images. To manage the size of these images, they are divided into smaller, overlapping tiles. To analyze the whole slide image, these tiles need to be accurately stitched back together.

To facilitate this, we have developed a Python class, `TileStitcher`, that leverages the power of QuPath, an open-source software platform for bioimage analysis, and the OpenJDK Java development kit. This class provides an efficient way to stitch together tiled images.

The `TileStitcher` class is a Python adaptation of a Groovy script by Pete Bankhead, available [here](https://gist.github.com/petebankhead/b5a86caa333de1fdcff6bdee72a20abe). The original Groovy script extracts the locations of the tiles from the baseline TIFF tags, stitches them together, and writes the stitched image as a pyramidal OME-TIFF. By reimplementing this in Python, we aim to provide a more accessible and easy-to-use tool for the digital pathology community.

In this tutorial, we will walk you through the process of using the `TileStitcher` class to collect, parse, stitch, and write large sets of tiled TIFF images. This class is particularly useful for dealing with tiled images from digital pathology experiments, where each image represents a small part of a larger whole. Let's get started!



## Prerequisites

Before you start using the `TileStitcher` class, it's crucial to have the necessary software installed and the environment correctly configured.

### Software Installation

`TileStitcher` requires QuPath and OpenJDK. Here are the steps to install them:

1\. **Install QuPath**  

   Download the latest version of QuPath from its [GitHub release page](https://github.com/qupath/qupath/releases), recommended version 0.4.3.

   ```wget https://github.com/qupath/qupath/releases/download/v0.4.3/QuPath-0.4.3-Linux.tar.xz
   tar -xvf QuPath-0.4.3-Linux.tar.xz```

   QuPath can also be installed using the `setupqupath` utility function during initialization. This function downloads QuPath from the specified URL. Be sure to adjust the QuPath home path as needed for your system.

2\. **Install OpenJDK 17**  

   OpenJDK 17 can be installed either through standard package managers or via Conda. If you're using Conda, you can install it from [Anaconda](https://anaconda.org/conda-forge/openjdk) as follows:

   ```conda install -c conda-forge openjdk~=17```

   Set the Java path according to your installation method. If you have set up your environment using PathML, set the Java path to `/opt/conda/envs/pathml`. Otherwise, adjust it to the appropriate path on your system.



### Environment Configuration

To use `TileStitcher`, we need to set the correct paths to the QuPath library and OpenJDK. For this, we need to add the paths to the environment variables `JAVA_HOME`, `CLASSPATH`, and `LD_LIBRARY_PATH`.

The `JAVA_HOME` environment variable should be set to the path where the JDK is installed.
The `CLASSPATH` environment variable should include paths to all the QuPath library jar files.
In the initialization of TileStitcher, these environment variables are used to start the Java Virtual Machine (JVM) and import the necessary QuPath classes.

## Best Practices and Considerations for Using the TileStitcher Module

### 1. JVM Session Management

The TileStitcher module utilizes jpype to manage the JVM sessions, a departure from the python-javabridge used in other parts of the package. This difference can cause conflicts when trying to run modules concurrently within the same Python environment. Hence, it is advisable to avoid running TileStitcher operations in the same notebook where python-javabridge dependent modules are running.

### 2. Restarting Kernel to Re-initialize JVM

The jpype does not allow the JVM to be restarted within the same Python session once it has been terminated. As a result, to run the TileStitcher module again or to switch back to modules that use python-javabridge, a kernel restart might be necessary.

### 3. Segregating Workflows

To mitigate potential conflicts, consider segregating workflows based on the JVM management package they depend on. It is recommended to use separate notebooks or scripts for operations involving TileStitcher and for those involving modules that are dependent on python-javabridge.



### Using TileStitcher

Ensure QuPath and JDK installations are complete before proceeding.

#### Initialization

The class is initialized with several parameters:

- `qupath_jarpath`: List of paths to QuPath JAR files.

- `java_path`: Custom path to Java installation. If set, `JAVA_HOME` will be overridden.

- `memory`: Allocated memory for the JVM (default: "40g").

- `bfconvert_dir`: Directory for Bio-Formats conversion tools.

During initialization, `TileStitcher` sets up the Java Virtual Machine (JVM) and imports necessary QuPath classes. It also includes error handling for Java path configurations and JVM startup issues.

#### JVM Startup

The `_start_jvm` method initiates the JVM with specified memory and classpath settings. It imports necessary QuPath classes upon successful startup, ensuring compatibility with Java 17.

In [12]:
import glob
import os
from pathml.preprocessing.tilestitcher import TileStitcher
from pathml.utils import setup_qupath


# Set the path to the JDK
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-17/"

# Use setup_qupath to get the QuPath installation path
qupath_home = setup_qupath('../../tools1/tools1/')

if qupath_home is not None:
    os.environ['QUPATH_HOME'] = qupath_home

    # Construct the path to QuPath jars based on qupath_home
    qupath_jars_dir = os.path.join(qupath_home, 'lib', 'app')
    qupath_jars = glob.glob(os.path.join(qupath_jars_dir, '*.jar'))
    qupath_jars.append(os.path.join(qupath_jars_dir, 'libopenslide-jni.so'))

    # Create an instance of TileStitcher
    stitcher = TileStitcher(qupath_jars)
else:
    print("QuPath installation not found. Please check the installation path.")



./tools/bftools/bfconvert ./tools/bftools/bf.sh
bfconvert version: Version: 7.0.1
Build date: 16 October 2023
VCS revision: 20e58cef1802770cc56ecaf1ef6f323680e4cf65
Setting Environment Paths
Java Home is already set
JVM was already started


In [13]:
import jpype

In [14]:
jpype.isJVMStarted()

False

### Image Stitching with TileStitcher

Once `TileStitcher` is initialized, it's capable of stitching together tiled images.

-   Method: `run_image_stitching`
-   Inputs:
    -   A list of TIFF files or a directory containing TIFF files.
    -   Output file path.
-   Optional Parameters:
    -   `downsamples`: Specify the number of downsample levels (e.g., `[1,4,8]`). Defaults to levels read from the tiles.
    -   `separate_series`: If set to `True`, it downloads bftools and extracts the base level image from the stitched image.

In [4]:
input_files = glob.glob("path/to/tiles/*.tif")`a
output_file = "path/to/output.ome.tif"
stitcher.run_image_stitching(input_files, output_file)


### **Demo**

In [1]:
import jpype

In [2]:
jpype.isJVMStarted(),jpype.getJVMVersion()

(False, (0, 0, 0))

In [3]:
import glob
import os
from pathml.preprocessing.tilestitcher import TileStitcher
from pathml.utils import setup_qupath


# Set the path to the JDK
os.environ["JAVA_HOME"] = "/opt/conda/envs/pathml"

# Use setup_qupath to get the QuPath installation path
qupath_home = setup_qupath('./tools/')

if qupath_home is not None:
    os.environ['QUPATH_HOME'] = qupath_home

    # Construct the path to QuPath jars based on qupath_home
    qupath_jars_dir = os.path.join(qupath_home, 'lib', 'app')
    qupath_jars = glob.glob(os.path.join(qupath_jars_dir, '*.jar'))
    qupath_jars.append(os.path.join(qupath_jars_dir, 'libopenslide-jni.so'))

    # Create an instance of TileStitcher
    stitcher = TileStitcher(qupath_jars)
else:
    print("QuPath installation not found. Please check the installation path.")



Using JAVA_HOME from environment variables.
Importing required qupath classes
Using JVM version: (0, 0, 0) from /opt/conda/envs/pathml/lib/jvm/lib/server/libjvm.so
JVM started successfully


In [4]:
jpype.isJVMStarted(),jpype.getJVMVersion()

(True, (17, 0, 3))

In [5]:
#Specify the folder path where the list of .tif files are present, here we are using a folder path that has single tif file for demo purposes.
infile_path= '../tests/testdata/tilestitching_testdata/'
outfile_path = './output/tile_stitching_demo.ome.tif'

In [6]:
import time

start= time.time()
# Run the image stitching process
stitcher.run_image_stitching(infile_path, outfile_path,downsamples=[1],separate_series=True)
end = time.time()

15:56:42.416 [main] [INFO ] q.l.i.s.b.BioFormatsServerOptions - Setting max Bio-Formats readers to 32
15:56:42.938 [main] [ERROR] q.l.i.s.o.OpenslideServerBuilder - Could not load OpenSlide native libraries
java.lang.UnsatisfiedLinkError: no openslide-jni in java.library.path: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib
	at java.base/java.lang.ClassLoader.loadLibrary(ClassLoader.java:2429)
	at java.base/java.lang.Runtime.loadLibrary0(Runtime.java:818)
	at java.base/java.lang.System.loadLibrary(System.java:1989)
	at org.openslide.OpenSlideJNI.<clinit>(OpenSlideJNI.java:55)
	at org.openslide.OpenSlide.<clinit>(OpenSlide.java:53)
	at qupath.lib.images.servers.openslide.OpenslideServerBuilder.<clinit>(OpenslideServerBuilder.java:90)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(Na